## ETL Pipelines ##
***

Call RAW Transaction.csv file, as well as output file path.

In [17]:
RAW_FILE_PATH = '../../data/raw/Transaction.csv'
OUTPUT_PATH = '../../data/'

Run schema definitions

In [18]:
%run '../../schema_definition/schema.py'

Import schema definitions

In [19]:
import sys  
sys.path.insert(0, '../../')

from schema_definition.schema import *

Import Pyspark related libraries

In [20]:
from pyspark.shell import spark
from pyspark.sql.types import *
from pyspark.sql import functions as F

Create a user defined function to add timestamp to ouput folder names.

In [21]:
from datetime import datetime

def file_processed_date():
    return datetime.now().strftime("%Y-%m-%d")

spark.udf.register("file_processed_date", file_processed_date, TimestampType())

21/10/16 12:33:24 WARN SimpleFunctionRegistry: The function file_processed_date replaced a previously registered function.


<function __main__.file_processed_date()>

This variable enables legacy TimeStamp datatype conversion

In [22]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

DataFrame[key: string, value: string]

### BRONZE ###
***

This paragraph will read/extract the data from the Transaction.csv file, force the schema to the default 17 columns and write onto a
tab separated csv file.

In [23]:
raw_df = spark.read.csv(RAW_FILE_PATH, sep=',', header=True, schema=bronze_schema, enforceSchema=True)
raw_df.write.format("parquet").mode("overwrite").save(OUTPUT_PATH + 'bronze/' + 'date' + '=' + file_processed_date())
raw_df.toPandas()

21/10/16 12:33:24 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 19, schema size: 17
CSV file: file:///Users/arturogonzalez/DataspellProjects/transactions_notebooks/data/raw/Transaction.csv
21/10/16 12:33:25 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 19, schema size: 17
CSV file: file:///Users/arturogonzalez/DataspellProjects/transactions_notebooks/data/raw/Transaction.csv


,AccountID,CODE,ImplementedDate,ActiveIndicator,AccountType,Service,BU,RequestDate,AccountStatus,StatusCode,Amount,Version,AgentID,Fibre,LastUpdatedDate,PropertyType,PostCode
0,15368.0,543,7/09/2017 0:00,1.0,B2B,ELE,NUOS,17/08/2017 0:00,PAID,8,3.94,v1_0,307508,EN121794032-N-5,7/09/2017 11:09,RESI,2587
1,23232.0,543,7/09/2017 0:00,1.0,B2B,ELE,NUOS,10/08/2017 0:00,PAID,7,126.96,v1_0,307508,EN121542407-N-12,7/09/2017 11:04,RESI,2587
2,232323.0,543,7/09/2017 0:00,1.0,B2B,ELE,NUOS,9/08/2017 0:00,PAID,6,15.43,v1_0,307508,EN121491293-N-9,7/09/2017 11:05,RESI,2587
3,307263.0,543,7/09/2017 0:00,1.0,B2B,ELE,NUOS,9/08/2017 0:00,PAID,6,3.49,v1_0,307508,EN121491293-N-13,7/09/2017 11:05,RESI,2587
4,415740.0,105,5/09/2017 0:00,1.0,B2B,ELE,NUOS,4/09/2017 0:00,PAID,8,3.31,v1_0,307312,EA983_54004586547-N-11,11/09/2017 10:44,RESI,2581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30315,10078.0,543,7/09/2017 0:00,1.0,B2B,ELE,NUOS,28/08/2017 0:00,PAID,7,6.92,v1_0,307510,EN122165150-N-15,7/09/2017 12:44,RESI,2587
30316,10095.0,543,7/09/2017 0:00,1.0,B2B,ELE,NUOS,29/08/2017 0:00,PAID,1,0.46,v1_0,307510,EN122228188-N-5,7/09/2017 12:48,RESI,2587
30317,10036.0,543,7/09/2017 0:00,1.0,B2B,ELE,NUOS,29/08/2017 0:00,PAID,7,8.53,v1_0,307510,EN122231542-N-9,7/09/2017 12:48,RESI,2587
30318,10053.0,543,7/09/2017 0:00,1.0,B2B,ELE,NUOS,29/08/2017 0:00,PAID,8,7.44,v1_0,307510,EN122225528-N-9,7/09/2017 12:48,RESI,2587


### SILVER ###
***

- Augment the data with a hash key.
- Filter out the questionable data.
- Apply DataTypes.
- Cleaned.


In [24]:
bronze_df = spark.read.parquet(OUTPUT_PATH + "bronze", mergeSchema=True)

augment = bronze_df.withColumn("HashKey", F.sha2(F.concat_ws("||", *bronze_df.columns), 256))

convert_to_timestamp = augment \
    .withColumn("AccountID",F.col("AccountID").cast(IntegerType())) \
    .withColumn("CODE",F.col("CODE").cast(IntegerType())) \
    .withColumn("ActiveIndicator",F.col("ActiveIndicator").cast(IntegerType())) \
    .withColumn("ImplementedDate", F.unix_timestamp("ImplementedDate", "d/MM/yyyy HH:mm").cast(TimestampType())) \
    .withColumn("RequestDate", F.to_timestamp("RequestDate", 'dd/MM/yyyy HH:mm').cast(TimestampType())) \
    .withColumn("StatusCode",F.col("StatusCode").cast(IntegerType())) \
    .withColumn("Amount",F.col("Amount").cast(DoubleType())) \
    .withColumn("AgentID",F.col("AgentID").cast(IntegerType())) \
    .withColumn("LastUpdatedDate", F.to_timestamp("LastUpdatedDate", 'dd/MM/yyyy HH:mm').cast(TimestampType())) \
    .withColumn("PostCode",F.col("PostCode").cast(IntegerType()))

drop_nulls = convert_to_timestamp.na.drop(subset=["AccountID"])
filter_out = drop_nulls.filter(~drop_nulls.Fibre.startswith('2.67E'))
drop_date_column  = filter_out.drop('date')
drop_date_column.write.format("parquet").mode("overwrite").save(OUTPUT_PATH + 'silver/' + 'date' + '=' + file_processed_date())
drop_date_column.toPandas()

,AccountID,CODE,ImplementedDate,ActiveIndicator,AccountType,Service,BU,RequestDate,AccountStatus,StatusCode,Amount,Version,AgentID,Fibre,LastUpdatedDate,PropertyType,PostCode,HashKey
0,15368,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-17,PAID,8,3.94,v1_0,307508,EN121794032-N-5,2017-09-07 11:09:00,RESI,2587,ae52557431727f096d968da7265c14abeb3bb3afa0fb4a...
1,23232,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-10,PAID,7,126.96,v1_0,307508,EN121542407-N-12,2017-09-07 11:04:00,RESI,2587,e4aad9f5903db9c97d5991d49f50e986b086e4c5343ca9...
2,232323,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-09,PAID,6,15.43,v1_0,307508,EN121491293-N-9,2017-09-07 11:05:00,RESI,2587,6e463c062b46ce88656b7971c482bb5489b9e012214848...
3,307263,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-09,PAID,6,3.49,v1_0,307508,EN121491293-N-13,2017-09-07 11:05:00,RESI,2587,6e2f04ff85422285e6cfe08508144b47262c77a590dee3...
4,415740,105,2017-09-05,1.0,B2B,ELE,NUOS,2017-09-04,PAID,8,3.31,v1_0,307312,EA983_54004586547-N-11,2017-09-11 10:44:00,RESI,2581,9057e3e8fc73f87b752b99e782400367c2ec2390358e72...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29636,10078,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-28,PAID,7,6.92,v1_0,307510,EN122165150-N-15,2017-09-07 12:44:00,RESI,2587,e6a92b6e6004c91e49608b9e9910abbb7644c3294acd6a...
29637,10095,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-29,PAID,1,0.46,v1_0,307510,EN122228188-N-5,2017-09-07 12:48:00,RESI,2587,a620f509f0657fd9a9e10f2b1db5645dd3bd518f1bcb2a...
29638,10036,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-29,PAID,7,8.53,v1_0,307510,EN122231542-N-9,2017-09-07 12:48:00,RESI,2587,39ba2d4d960ac2baf636a187ad2e18eea7470aa5f1d94e...
29639,10053,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-29,PAID,8,7.44,v1_0,307510,EN122225528-N-9,2017-09-07 12:48:00,RESI,2587,7a531111b3b8f0efe538d47cf521d2fd95475824e623da...


### GOLD ###
***

- Presentation layer.

In [25]:
silver_df = spark.read.parquet(OUTPUT_PATH + "silver", mergeSchema=True)
drop_date_column  = silver_df.drop('date')
# drop_date_column.coalesce(1).write.option("maxRecordsPerFile", 1000).json(OUTPUT_PATH + 'gold', mode='overwrite')
drop_date_column.write.format("parquet").mode("overwrite").save(OUTPUT_PATH + 'gold/' + 'date' + '=' + file_processed_date())
drop_date_column.toPandas()

,AccountID,CODE,ImplementedDate,ActiveIndicator,AccountType,Service,BU,RequestDate,AccountStatus,StatusCode,Amount,Version,AgentID,Fibre,LastUpdatedDate,PropertyType,PostCode,HashKey
0,15368,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-17,PAID,8,3.94,v1_0,307508,EN121794032-N-5,2017-09-07 11:09:00,RESI,2587,ae52557431727f096d968da7265c14abeb3bb3afa0fb4a...
1,23232,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-10,PAID,7,126.96,v1_0,307508,EN121542407-N-12,2017-09-07 11:04:00,RESI,2587,e4aad9f5903db9c97d5991d49f50e986b086e4c5343ca9...
2,232323,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-09,PAID,6,15.43,v1_0,307508,EN121491293-N-9,2017-09-07 11:05:00,RESI,2587,6e463c062b46ce88656b7971c482bb5489b9e012214848...
3,307263,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-09,PAID,6,3.49,v1_0,307508,EN121491293-N-13,2017-09-07 11:05:00,RESI,2587,6e2f04ff85422285e6cfe08508144b47262c77a590dee3...
4,415740,105,2017-09-05,1.0,B2B,ELE,NUOS,2017-09-04,PAID,8,3.31,v1_0,307312,EA983_54004586547-N-11,2017-09-11 10:44:00,RESI,2581,9057e3e8fc73f87b752b99e782400367c2ec2390358e72...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29636,10078,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-28,PAID,7,6.92,v1_0,307510,EN122165150-N-15,2017-09-07 12:44:00,RESI,2587,e6a92b6e6004c91e49608b9e9910abbb7644c3294acd6a...
29637,10095,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-29,PAID,1,0.46,v1_0,307510,EN122228188-N-5,2017-09-07 12:48:00,RESI,2587,a620f509f0657fd9a9e10f2b1db5645dd3bd518f1bcb2a...
29638,10036,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-29,PAID,7,8.53,v1_0,307510,EN122231542-N-9,2017-09-07 12:48:00,RESI,2587,39ba2d4d960ac2baf636a187ad2e18eea7470aa5f1d94e...
29639,10053,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-29,PAID,8,7.44,v1_0,307510,EN122225528-N-9,2017-09-07 12:48:00,RESI,2587,7a531111b3b8f0efe538d47cf521d2fd95475824e623da...
